We will implement a Gibbs sampling algortihm to sample the factors from the conditional posterior distrubtions in the MVGMF algorithm to factorise our matrix. We will call this algortihm Bayesian Gaussian Matrix Factorisation (BGMF) for now.

In [17]:
import numpy as np
import matplotlib.pyplot as plt

In [18]:
def calculate_nmse(Y, Y_pred):
    numerator = np.linalg.norm(Y - Y_pred, 'fro') ** 2
    denominator = np.linalg.norm(Y, 'fro') ** 2
    nmse = numerator / denominator
    return nmse

In [19]:
def matrix_normal_sample(M, U, V):
    return np.random.multivariate_normal(M.ravel(), np.kron(V, U)).reshape(M.shape)

In [31]:
def BGMF(Y, k, r, T):
    """
    Perform Bayesian Gaussian Matrix Factorization (BGMF) using Gibbs sampling.

    Parameters:
    - Y (numpy.ndarray): The input matrix of shape (m, n) to be factorized.
    - k (float): A constant for the variate matrix initialization.
    - r (int): The dimensionality of the factorization.
    - T (int): The number of Gibbs sampling iterations.

    Returns:
    - C (numpy.ndarray): The factor matrix C of shape (m, r).
    - X (numpy.ndarray): The factor matrix X of shape (r, n).
    """
    m, n = Y.shape
    sigma_2 = 1
    #Initialise the matrices
    V = k * np.eye(r)
    W = k * np.eye(r)
    C_prev = np.random.normal(0, 1, size=(m, r))
    X_prev = np.random.normal(0, 1, size=(r, n))
    
    for t in range(T):
        #Note is it ^T or ^-T?
        C_mean = C_prev + (Y - C_prev @ X_prev) @ \
                    np.linalg.inv(sigma_2*np.eye(n) + X_prev.T@V@X_prev).T @ X_prev.T @ V.T
        
        C = matrix_normal_sample(C_mean, np.eye(m), V)
        
        X_mean = X_prev + W @ C.T @ np.linalg.inv(sigma_2*np.eye(m) + C @ W @ C.T) @ (Y - C @ X_prev)
        
        X = matrix_normal_sample(X_mean, W, np.eye(n))
        
        C_prev = C
        X_prev = X
        
        V *= k
        W *= k
        
    return C, X

In [66]:
m = 25
r = 3
n = 5

C = np.random.normal(0, 1, size=(m, r))
X = np.random.normal(0, 1, size=(r, n))

Y = C @ X

C_pred, X_pred = BGMF(Y, 0.95, r, 500)
Y_pred = C_pred @ X_pred

calculate_nmse(Y, Y_pred)

0.15756254583080653